## Dependencies required are pyaudio, speechrecognition, playsound

In [7]:
#vital imports and one time access files

import os, pyaudio, wave, time, csv, pandas as pd, speech_recognition as sr
# from gtts import gTTS
from playsound import playsound
from threading import Thread
recognizer=sr.Recognizer()
    

probable_ans=pd.read_csv("csv/probable_answers.csv", keep_default_na=False)
temp=probable_ans.iloc[:,1:]
ans=temp.to_numpy().tolist()

bool_map=pd.read_csv("csv/bool_map.csv", header=None)
mapping=bool_map.to_numpy().tolist()

questions=probable_ans.iloc[:,:1].to_numpy().tolist()
no_of_questions=(len([name for name in os.listdir('questions/')]))


def recognize(filename, lang="ne-NP"):
    my_file=sr.AudioFile(filename)
    with my_file as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.record(source)
    try:
        output=recognizer.recognize_google(audio, language=lang, show_all=True)
    except sr.RequestError:
        output="API unavailable or did not response"
    except sr.UnknownValueError:
        output="Did not recognize the audio"
    return output


def record_and_save(num, seconds=5):
    print("Q. NO. "+str(num+1)+" "+questions[num][0])
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 2
    fs = 44100
    filename = folder+"/"+str(num)+".wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []


    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)


    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [40]:
#record here

from random import randint
phone_num=randint(9800000000,9869999999)
print("Recording with "+str(phone_num))
folder="recorded/"+str(phone_num)
os.mkdir(folder)

for i in range(0, no_of_questions):
    temp="questions/"+str(i)+".mp3"
    seconds=5
    if(i==13 or i==14):
        seconds=7
    #thread creation
    t1=Thread(target=playsound, args=(temp,))
    t2=Thread(target=record_and_save, args=(i,seconds,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()

Recording with 9820909358
Q. NO. 0
Q. NO. 1
Q. NO. 2
Q. NO. 3
Q. NO. 4
Q. NO. 5
Q. NO. 6
Q. NO. 7
Q. NO. 8
Q. NO. 9
Q. NO. 10
Q. NO. 11
Q. NO. 12
Q. NO. 13
Q. NO. 14


In [18]:
#recognize here
output=[]
for i in range(0, no_of_questions):
    temp=[]
    filename="recorded/"+str(phone_num)+"/"+str(i)+".wav"
    converted=recognize(filename)
    if(len(converted)==0):
        output.append("err")
    else:
        for a in converted['alternative']:
            temp.append(a['transcript'])
        output.append(temp)

        
        
#append to csv here
import numpy as np
results=[phone_num]
results_bool=[phone_num]

for i in range(0, no_of_questions):
    c = [value for value in ans[i] if value in output[i]]
    if output[i]=="err":
        results.append("err")
        results_bool.append("err")
    else:
        if len(c)>0:
            results.append(c[0])
            #check if c[0] is 1 or 0
            ind=mapping[0].index(c[0])
            results_bool.append(mapping[1][ind])
        else:
            results.append("*")
            results_bool.append("*")

with open(r'csv/results.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(results)
    

with open(r'csv/results_bool.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(results_bool)

print("Phone number "+str(phone_num)+"\n")
for i in range(0, no_of_questions):
    print("Q.No. "+str(i+1)+" "+questions[i][0]+"  "+results[i+1])
    print()
    
#     print("Q.No. "+str(i+1)+" "+questions[i][0])
#     print(output[i])
    

# print("Final Results")
# print(results)
# print()
# print("Final Results in 1 and 0")
# print(results_bool)

Phone number 9820909358

Q.No. 1 नाक बाट सिङान बगिरहेको छ?  छ

Q.No. 2 ज्वरो आएको छ? (१०० F+)  छैन

Q.No. 3 सुक्खा खोकी लागेको छ?   छैन

Q.No. 4 धेरै थकाइ लागेको छ ?   छैन

Q.No. 5 खोक्दा खकार आउछ ?   आउछ

Q.No. 6 सास फेर्न गार्हो भएको छ?   छैन

Q.No. 7 घाँटी दुखेको छ ?   छ

Q.No. 8 टाउको दुखेको छ ?   छ

Q.No. 9 जिउ दुखेको छ ?   छैन

Q.No. 10 जिउ सिरिङ्ग सिरिङ्ग  भएको छ ?   छैन

Q.No. 11 वाक वाकी र बान्ता भएको छ ?   छैन

Q.No. 12 नाक थुनिएको छ ?   छ

Q.No. 13 पखाला लागेको छ ?   छैन

Q.No. 14 बिदेश बाट फर्केको वा कोरोना सन्क्रमित ब्यक्तिसँग सम्पर्कमा आउनु भएको छ ?   थाहा छैन

Q.No. 15 पछील्लो १४ दिनमा बिदेश भ्रमन गर्नुभएको छ ?   *



In [12]:
pd.read_csv("csv/results.csv", header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,9869231782,छ,छैन,*,*,*,err,*,छ,*,*,छैन,छैन,छैन,*,छैन
1,9869231782,छ,छैन,*,*,*,err,*,छ,*,*,छैन,छैन,छैन,*,छैन
2,9869231782,छ,छैन,*,*,*,err,*,छ,*,*,छैन,छैन,छैन,*,छैन
3,9818991368,err,छैन,*,err,*,छैन,*,*,छ,*,*,err,*,err,err
4,9834765726,*,छैन,बगेको छ,छैन,err,छैन,छ,*,छैन,*,err,छ,छैन,err,*
5,9834765726,*,छैन,बगेको छ,छैन,err,छैन,छ,*,छैन,*,err,छ,छैन,err,*
6,9834765726,*,छैन,बगेको छ,छैन,err,छैन,छ,*,छैन,*,err,छ,छैन,err,*
7,9834765726,*,छैन,लागेको छ,छैन,err,छैन,छ,*,छैन,*,err,छ,छैन,err,*
8,9803693386,बगिरहेको छ,छैन,लागेको छ,लागेको छ,err,छैन,err,छ,err,छैन,err,*,छैन,err,छैन
9,9841622277,छ,err,*,err,आउछ,छैन,err,छैन,err,छ,भएको छ,err,err,*,छैन


In [13]:
pd.read_csv("csv/results_bool.csv", header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,9834765726,*,0,1,0,err,0,1,*,0,*,err,1,0,err,*
1,9834765726,*,0,1,0,err,0,1,*,0,*,err,1,0,err,*
2,9803693386,1,0,1,1,err,0,err,1,err,0,err,*,0,err,0
3,9841622277,1,err,*,err,1,0,err,0,err,1,1,err,err,*,0
4,9867973643,1,err,1,err,0,err,*,0,1,0,0,err,1,*,0
5,9820909358,1,0,0,0,1,0,1,1,0,0,0,1,0,NaN,*
6,9820909358,1,0,0,0,1,0,1,1,0,0,0,1,0,NaN,*
7,9820909358,1,0,0,0,1,0,1,1,0,0,0,1,0,NaN,*


In [ ]:
# questions = ["के तपाईको टाउको दुख्छ", "के तपाइलाई ज्वरो छ ", "के तपाई को नाक बग्छ ", "के तपाइलाई सास फेर्न समस्या छ "]
# for i in range(0, len(questions)):
#     speech = gTTS(text[i], lang = "en-IN", slow = False)
#     name="questions/"+str(i)+".mp3"
#     speech.save(name)

# def listen_and_recognize(lang="ne-NP"):
#     mic = sr.Microphone(device_index=1)
#     with mic as source:
#         recognizer.adjust_for_ambient_noise(source)
#         audio = recognizer.listen(source)
#     try:
#         output=recognizer.recognize_google(audio, language=lang)
#     except sr.RequestError:
#         output="API unavailable"
#     except sr.UnknownValueError:
#         output="recognition err"
#     return output

# def listen_and_save(num, lang="ne-NP"):
#     mic = sr.Microphone(device_index=1)
#     with mic as source:
#         audio = recognizer.listen(source)
#         filename="recorded/rec_"+str(num)+".wav"
#         with open(filename, "wb") as f:
#             f.write(audio.get_wav_data())

# #listen and recognize here
# for i in range(0, 4):
#     temp="questions/text_"+str(i)+".mp3"
#     playsound(temp)
#     print(listen_and_recognize(i))
    
# #listen and save here
# for i in range(0, 4):
#     temp="questions/text_"+str(i)+".mp3"
#     playsound(temp)
#     print(listen_and_save(i))